### Import Necessary Libraries

In [26]:
# Text Embedding Model
from sentence_transformers import SentenceTransformer

# Wikipedia API to fetch text
from wikipediaapi import Wikipedia

## To display text in nicer format in console or cell
import textwrap

# For vector operations
import numpy as np

# LLM
import openai
from openai import OpenAI

from dotenv import load_dotenv

### Load the Embedding Model

In [2]:
model = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

### Fetch Text Content from Wikipedia

In [5]:
wiki = Wikipedia('RAGBot/0.0', 'en')
doc = wiki.page('Muhammad_Yunus').text
paragraphs = doc.split('\n\n') # chunking


In [6]:
for i, p in enumerate(paragraphs):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")


-----------------------------------------------------------------
Muhammad Yunus (born 28 June 1940) is a Bangladeshi entrepreneur, banker, economist, politician, and
civil society leader who has been serving as the Chief Adviser of the interim government of
Bangladesh since 8 August 2024. Yunus was awarded the Nobel Peace Prize in 2006 for founding the
Grameen Bank and pioneering the concepts of microcredit and microfinance. Yunus has received several
other national and international honors, including the United States Presidential Medal of Freedom
in 2009 and the Congressional Gold Medal in 2010. Yunus is one of just seven individuals who have
been awarded the Nobel Peace Prize, the United States Presidential Medal of Freedom, and the United
States Congressional Gold Medal. In 2012, Yunus became Chancellor of Glasgow Caledonian University
in Scotland, a position he held until 2018. Previously, he was a professor of economics at
Chittagong University in Bangladesh. He published severa

In [10]:
len(paragraphs)

35

### Embed the Document

Turning the paragraphs into word embeddings 

In [7]:
docs_embed = model.encode(paragraphs, normalize_embeddings=True)

In [8]:
docs_embed.shape

(35, 768)

In [12]:
docs_embed[0]

array([ 1.91694759e-02, -2.69589331e-02,  1.04318727e-02, -2.70635653e-02,
       -4.21567857e-02, -7.71854678e-03,  5.74812144e-02,  5.77369966e-02,
        3.16994675e-02, -3.60584892e-02,  4.18407209e-02,  3.19251381e-02,
        2.97827795e-02, -3.81309949e-02, -4.77523264e-03, -1.81123521e-02,
        2.39235461e-02, -4.39292565e-03, -1.49219790e-02,  1.07248975e-02,
       -2.00983137e-02, -2.68920455e-02, -2.36109160e-02,  2.09223814e-02,
        6.27344241e-03,  1.29387341e-02, -1.16277458e-02,  4.59472351e-02,
       -4.65470552e-02, -7.52445485e-04, -3.84207256e-02, -1.14051194e-03,
        3.64743434e-02, -1.97858252e-02, -1.79965440e-02, -2.69578472e-02,
       -2.03525759e-02,  1.57506652e-02, -2.34666448e-02,  2.65341178e-02,
       -8.46144930e-03,  4.74700611e-03,  1.17120706e-02,  8.31734017e-03,
       -3.00525571e-03,  5.86115047e-02,  1.42740691e-02, -3.20817754e-02,
        8.89801979e-03, -5.40683009e-02, -2.85662916e-02,  5.06849214e-02,
       -3.74441557e-02, -

In [13]:
query = "When did Dr.Yunus won the Nobel peace prize?"
query_embed = model.encode(query, normalize_embeddings=True)


In [14]:
query_embed.shape

(768,)

### Find the Closest Paragraph to the Query

In [16]:
similarities = np.dot(docs_embed, query_embed.T)

In [17]:
similarities.shape

(35,)

In [18]:
top_3_idx = np.argsort(similarities, axis=0)

In [19]:
top_3_idx

array([24, 30, 33, 31,  4, 18, 14, 34, 16, 28, 32, 23, 29, 13, 21,  3, 15,
       27, 22, 20, 12,  1, 17, 25, 11, 19,  9,  2, 26, 10,  8,  6,  7,  0,
        5])

In [20]:
top_3_idx[-3:]

array([7, 0, 5])

In [21]:
top_3_idx[-3:][::-1]

array([5, 0, 7])

In [22]:
top_3_idx = np.argsort(similarities, axis=0)[-3:][::-1].tolist()

In [23]:
most_similar_documents = [paragraphs[idx] for idx in top_3_idx]

In [24]:
CONTEXT = ""
for i, p in enumerate(most_similar_documents):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")
  CONTEXT += wrapped_text + "\n\n"

-----------------------------------------------------------------
Recognition Yunus was awarded the 2006 Nobel Peace Prize, along with Grameen Bank, for their efforts
to create economic and social development:
-----------------------------------------------------------------
-----------------------------------------------------------------
Muhammad Yunus (born 28 June 1940) is a Bangladeshi entrepreneur, banker, economist, politician, and
civil society leader who has been serving as the Chief Adviser of the interim government of
Bangladesh since 8 August 2024. Yunus was awarded the Nobel Peace Prize in 2006 for founding the
Grameen Bank and pioneering the concepts of microcredit and microfinance. Yunus has received several
other national and international honors, including the United States Presidential Medal of Freedom
in 2009 and the Congressional Gold Medal in 2010. Yunus is one of just seven individuals who have
been awarded the Nobel Peace Prize, the United States Presidential Med

### Getting the response from LLM

In [35]:
_ = load_dotenv()

client = OpenAI()

In [32]:
query = "When did Dr.Yunus won the Nobel peace prize?"
prompt = f"""Use the following CONTEXT to answer the QUESTION at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

CONTEXT: {CONTEXT}
QUESTION: {query}

"""

In [33]:
print(prompt)

Use the following CONTEXT to answer the QUESTION at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

CONTEXT: Recognition Yunus was awarded the 2006 Nobel Peace Prize, along with Grameen Bank, for their efforts
to create economic and social development:

Muhammad Yunus (born 28 June 1940) is a Bangladeshi entrepreneur, banker, economist, politician, and
civil society leader who has been serving as the Chief Adviser of the interim government of
Bangladesh since 8 August 2024. Yunus was awarded the Nobel Peace Prize in 2006 for founding the
Grameen Bank and pioneering the concepts of microcredit and microfinance. Yunus has received several
other national and international honors, including the United States Presidential Medal of Freedom
in 2009 and the Congressional Gold Medal in 2010. Yunus is one of just seven individuals who have
been awarded the Nobel Peace Prize, the United States Presidential Medal of Freedom, and the United
Stat

In [36]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": prompt},
  ]
)

In [42]:
response.choices[0].message.content

'Dr. Yunus won the Nobel Peace Prize in 2006.'